# How to Define Custom Gates
`PauliPropagation.jl` is extensible and allows you to define your own gates. Depending on how much you can or want to code, you can definte a gate that _works_ or one that is as fast as it gets. Here will see what you need to define.

In [1]:
using PauliPropagation

Let us start by defining a `SWAP` gate. It is sub-typing from `StaticGate`, which denotes that it does not take any variable parameters at propagation time. It always acts the same.

In [2]:
struct CustomSWAPGate <: StaticGate
    qinds::Tuple{Int, Int}  # The two sites to be swapped
end

The action of a `SWAP` gate on a Pauli string is that it swaps the Paulis on two sites. We can now define a function `apply` which receives these 3 arguments in this order, as well as potential `kwargs`: `apply(gate::YourGate, pstr, coefficient; kwargs...)`. We can ignore `kwargs` for now, but you can use them to pass arguments from the top level down to your function. If your custom gate is parametrized, then you have you sub-type `ParametrizedGate` instead of `StaticGate`, and the `apply` function
will receive a fourth argument, which is the parameter: `apply(gate::YourGate, pstr, coefficient, theta; kwargs...)`.

This is how you can define `SWAP`:

In [3]:
function PauliPropagation.apply(gate::CustomSWAPGate, pstr, coefficient; kwargs...)
    # get the Pauli on the first site
    pauli1 = getpauli(pstr, gate.qinds[1])
    # get the Pauli on the second site
    pauli2 = getpauli(pstr, gate.qinds[2])
    
    # set the Pauli on the first site to the second Pauli
    pstr = setpauli(pstr, pauli2, gate.qinds[1])
    # set the Pauli on the second site to the first Pauli
    pstr = setpauli(pstr, pauli1, gate.qinds[2])

    return pstr, coefficient
end

This is it, really.

Now set up the simulation. 25 qubits in a 5 by 5 grid.

In [4]:
nx = 5
ny = 5
nq = nx * ny

topology = get2dtopology(nx, ny);

`nl` layers of a circuit consisting of `RX` and `RZZ` Pauli rotations.

In [5]:
nl = 3
base_circuit = tfitrottercircuit(nq, nl; topology=topology);
nparams = countparameters(base_circuit)

195

Define our observable as $ Z_7 Z_{13} $.

In [6]:
pstr = PauliString(nq, [:Z, :Z], [7, 13])

PauliString(nqubits: 25, 1.0 * IIIIIIZIIIIIZIIIIIII...)

Circuit parameters with a random seed.

In [7]:
using Random
Random.seed!(42)
thetas = randn(nparams);

For this notebook, we will use a minimum coefficient threshold. The results are still almost exact.

In [8]:
min_abs_coeff = 5e-3

0.005

Now add a 1D line of SWAP gates after the first and second layer of gates in the base circuit.

In [9]:
nparams_per_layer = Int(length(base_circuit)/nl)

65

In [10]:
ourSWAP_circuit = deepcopy(base_circuit);
# first the second layer so the insertion indices don't change 
for qind in 1:(nq-1)
    insert!(ourSWAP_circuit, 2*nparams_per_layer, CustomSWAPGate((qind, qind+1)))
end
for qind in 1:(nq-1)
    insert!(ourSWAP_circuit, nparams_per_layer, CustomSWAPGate((qind, qind+1)))
end

Run the circuit

In [11]:
@time ourSWAP_psum = propagate(ourSWAP_circuit, pstr, thetas; min_abs_coeff=min_abs_coeff)

  0.477635 seconds (479.11 k allocations: 31.450 MiB, 99.57% compilation time)


PauliSum(nqubits: 25, 576 Pauli terms:
 0.0053683 * IIIIYZIIIXYIIIZIIIII...
 -0.010561 * IIIIIYIIIXXYZIIIZIII...
 -0.0065309 * IIIZXYIIIYYZIIIIIIII...
 -0.0069415 * ZIIZYXZIIZXXZIIIZIII...
 0.0082654 * IIIZXZIIIXXYZIZIZIII...
 0.080065 * IIIZYIIIIIYIIIIIIIII...
 -0.0056512 * IIIIZXIIIXXXZIZZXIII...
 -0.0060663 * IIIIZZIIIIXYZIIXIIII...
 -0.01727 * ZIIZXXIIIXZIIIZIIIII...
 -0.0085018 * IIIIIZIIIXYIIIIIIIII...
 0.0093524 * IIIIIXIIIYXXZIZZXIII...
 -0.03761 * IIIZXZIIIZZIIIIIIIII...
 -0.01582 * IIIZYZIIIIYIIIIIIIII...
 -0.0059443 * IIIZXXIIIZZXZIIIZIII...
 -0.0063894 * IIIZXIIIIZYYZIIIZIII...
 0.022245 * IIIZYIIIIZYZIIIIIIII...
 -0.024117 * IIIIIZIIIYXZIIZIIIII...
 -0.0075649 * IIIIZZZIIXXXZIZZXIII...
 -0.013565 * IIIZYIIIIZZZIIIIIIII...
 0.0070561 * IIIZXXIIIXXXZIZIZIII...
  ⋮)

Overlap with the zero-state

In [12]:
overlapwithzero(ourSWAP_psum)

-0.7211301948203008

This looks okay, but is it correct? One thing you may have noticed is that `SWAP` is a `Clifford` operation, i.e., one that takes one Pauli to exactly one other Pauli. We actually have that in our package so we can easily compare.

In [13]:
cliffSWAP_circuit = deepcopy(base_circuit);
for qind in 1:(nq-1)
    insert!(cliffSWAP_circuit, 2*nparams_per_layer, CliffordGate(:SWAP, (qind, qind+1)))
end
for qind in 1:(nq-1)
    insert!(cliffSWAP_circuit, nparams_per_layer, CliffordGate(:SWAP, (qind, qind+1)))
end

In [14]:
@time cliffSWAP_psum = propagate(cliffSWAP_circuit, pstr, thetas; min_abs_coeff=min_abs_coeff);

  0.071685 seconds (63.76 k allocations: 4.651 MiB, 97.44% compilation time)


Are the results the same?

In [15]:
overlapwithzero(cliffSWAP_psum)

-0.7211301948203008

In [16]:
cliffSWAP_psum == ourSWAP_psum

true

Yes!

We can also benchmark the performance.

In [17]:
using BenchmarkTools

In [18]:
@btime propagate($ourSWAP_circuit, $pstr, $thetas; min_abs_coeff=$min_abs_coeff);

  1.628 ms (1081 allocations: 471.59 KiB)


In [19]:
@btime propagate($cliffSWAP_circuit, $pstr, $thetas; min_abs_coeff=$min_abs_coeff);

  1.690 ms (1081 allocations: 471.59 KiB)


No downside at all from defining our custom gate. How? This is because the `apply` function for this gate is *type stable*! Type stability is absolutely crucial in Julia, and codes live and die by it.

In [20]:
@code_warntype apply(CustomSWAPGate((7, 8)), pstr.term, 0.0, 1.0)

MethodInstance for PauliPropagation.apply(::CustomSWAPGate, ::UInt64, ::Float64, ::Float64)
  from apply(gate::SG, pstr, coeff, theta; kwargs...) where SG<:StaticGate @ PauliPropagation ~/.julia/dev/PauliPropagation/src/Propagation/generics.jl:164
Static Parameters
  SG = CustomSWAPGate
Arguments
  #self#::Core.Const(PauliPropagation.apply)
  gate::CustomSWAPGate
  pstr::UInt64
  coeff::Float64
  theta::Float64
Body::Tuple{UInt64, Float64}
1 ─ %1 = Core.NamedTuple()::Core.Const(NamedTuple())
│   %2 = Base.pairs(%1)::Core.Const(Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}())
│   %3 = PauliPropagation.:(var"#apply#89")(%2, #self#, gate, pstr, coeff, theta)::Tuple{UInt64, Float64}
└──      return %3



All blue means that everything is great! If correctly implemented, `apply` will be type stable if it returns a known number of Pauli and coefficient pairs. Here it is just 1 because it is a Clifford gate.

Onto an example of a gate that can _split_ a Pauli string into two: The `T` gate.

In [21]:
struct CustomTGate <: StaticGate
    qind::Int
end

A `T` gate is a non-Clifford gate that commutes with `I` and `Z`, splits `X` into `cos(π/4)X - sin(π/4)Y`, and `Y` into `cos(π/4)Y + sin(π/4)X`. 

Let's write the code for that.

In [22]:
function PauliPropagation.apply(gate::CustomTGate, pstr, coefficient; kwargs...)
    # get the Pauli on the site `gate.qind`
    pauli = getpauli(pstr, gate.qind)
    
    if pauli == 0 || pauli == 3  # I or Z commute
        return pstr, coefficient     
    end
    
    if pauli == 1 # X goes to X, -Y
        new_pauli = 2  # Y
        # set the Pauli
        new_pstr = setpauli(pstr, new_pauli, gate.qind)
        # adapt the coefficients
        new_coefficient = -1 * coefficient * sin(π/4)
        coefficient_prime = coefficient * cos(π/4)
        
    else # Y goes to Y, X
        new_pauli = 1  # X
        # set the Pauli
        new_pstr = setpauli(pstr, new_pauli, gate.qind)
        # adapt the coefficients
        new_coefficient = coefficient * sin(π/4)
        coefficient_prime = coefficient * cos(π/4)
    end
    
    return pstr, coefficient_prime, new_pstr, new_coefficient
    
end

Insert a layer of `TGate`s after the first layer of the base circuit.

In [23]:
ourT_circuit = deepcopy(base_circuit);
for qind in 1:nq
    insert!(ourT_circuit, 2*nparams_per_layer, CustomTGate(qind))
end
for qind in 1:nq
    insert!(ourT_circuit, nparams_per_layer, CustomTGate(qind))
end

And run:

In [24]:
@time ourT_psum = propagate(ourT_circuit, pstr, thetas; min_abs_coeff=min_abs_coeff)

  0.064375 seconds (168.62 k allocations: 8.361 MiB, 87.55% compilation time)


PauliSum(nqubits: 25, 1702 Pauli terms:
 0.0058949 * IZIIIYXYIIIZXIIIIZII...
 0.013378 * IIIIIIYIIIIZXIIIIZII...
 -0.032827 * IZIIIZYZIIIZYZIIIZII...
 0.0058029 * IZZIIIIYIIIZXZIIIZII...
 0.0054978 * IZIIIYXZIIIXXIIIZZII...
 -0.01082 * IIIIIZXIIIIZYZIIIYZI...
 -0.0066769 * IZIIIYXIIIIZXIIIIYZI...
 -0.024172 * IIIIIZXIIIIYIIIIZZII...
 -0.014213 * IIIIIIXIZIIZYYIIIYII...
 -0.0073527 * IIIIIIIIZIIYXYIIZXII...
 0.0075631 * IIIIIIIIIIIXXZIIIYZI...
 -0.0090057 * IZIIIIZYIIIZXIIIIZII...
 -0.053549 * IIIIIZXZIIIZXZIIIZII...
 0.01401 * IZIIIYXYIIIZYZIIIZII...
 0.0097479 * IIIIIIXIZIIIXYIIIZZI...
 -0.02131 * IIIIIIYIIIIYXZIIIZII...
 0.019155 * IIIIIIXIIIIZXZIIIXZI...
 -0.0095632 * IIIIIIZZZIIXXXZIZZZI...
 0.015053 * IIIIIIYIZIIXXYIIZYII...
 0.0063405 * IIIIIIYIIIIYIIIIZYZI...
  ⋮)

In [25]:
overlapwithzero(ourT_psum)

0.3326289935884039

But did it work? Again, we have an implementation of a `TGate` in our library. In case you are interested, we currently implement `T` gates as Pauli `Z` rotations at an angle of `π/4`. Let's compare to that.

In [26]:
frozenZ_circuit = deepcopy(base_circuit);
for qind in 1:nq
    insert!(frozenZ_circuit, 2*nparams_per_layer, TGate(qind))
end
for qind in 1:nq
    insert!(frozenZ_circuit, nparams_per_layer, TGate(qind))
end

If you call `PauliGate(:Z, qind, parameter)`, this will create a so-called `FrozenGate` wrapping the parametrized `PauliGate`, with a fixed `parameter` at the time of circuit construction.

Run it and compare

In [27]:
@time frozenZ_psum = propagate(frozenZ_circuit, pstr, thetas; min_abs_coeff=min_abs_coeff);

  0.022177 seconds (11.34 k allocations: 2.497 MiB, 29.16% gc time, 42.21% compilation time)


In [28]:
overlapwithzero(frozenZ_psum)

0.3326289935884039

In [29]:
frozenZ_psum == ourT_psum

true

It works! But is it optimal?

In [30]:
using BenchmarkTools

In [31]:
@btime propagate($ourT_circuit, $pstr, $thetas;min_abs_coeff=$min_abs_coeff);

  7.675 ms (104334 allocations: 4.13 MiB)


In [32]:
@btime propagate($frozenZ_circuit, $pstr, $thetas; min_abs_coeff=$min_abs_coeff);

  6.075 ms (1158 allocations: 1.79 MiB)


No, because `apply` for the `CustomTGate` is not type-stable.

In [33]:
@code_warntype apply(CustomTGate(7), pstr.term, 0.0)

MethodInstance for PauliPropagation.apply(::CustomTGate, ::UInt64, ::Float64)
  from apply(gate::CustomTGate, pstr, coefficient; kwargs...) @ Main In[22]:1
Arguments
  #self#::Core.Const(PauliPropagation.apply)
  gate::CustomTGate
  pstr::UInt64
  coefficient::Float64
Body::Union{Tuple{UInt64, Float64}, Tuple{UInt64, Float64, UInt64, Float64}}
1 ─ %1 = Core.NamedTuple()::Core.Const(NamedTuple())
│   %2 = Base.pairs(%1)::Core.Const(Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}())
│   %3 = Main.:(var"#apply#2")(%2, #self#, gate, pstr, coefficient)::Union{Tuple{UInt64, Float64}, Tuple{UInt64, Float64, UInt64, Float64}}
└──      return %3



It either returns a tuple `Tuple{UInt64, Float64}` of length 2 or a tuple `Tuple{UInt64, Float64, UInt64, Float64}` of length 4. When this is the case, you may want to define some lower-level function under `propagate` for optimal performance. This is how we would do it. Yellow `@code_warntype` output means it might be okay (it is not that much slower after all), but be wary of red.

To avoid such type instabilities, we can overload a slightly higher level function `applyandadd!()`, which does the job of `apply()`, but as the name hints, also adds the created Pauli strings to the propagating Pauli sum. We can practically copy-paste the code from `apply()`, but the only difference being that we don't return anything, but `add!()` the Pauli strings to the `output_psum`. Be mindful of the fact that the function signature needs to be exactly like this. Even though you might not need a parameter `theta`, it needs to be received by your function.

In [34]:
function PauliPropagation.applyandadd!(gate::CustomTGate, pstr, coefficient, theta, output_psum; kwargs...)
    
    pauli = getpauli(pstr, gate.qind)
    
    if pauli == 0 || pauli == 3  # I or Z commute
        add!(output_psum, pstr, coefficient)
        return
    end

    if pauli == 1 # X goes to X, -Y
        new_pauli = 2  # Y
        new_pstr = setpauli(pstr, new_pauli, gate.qind)
        new_coefficient = -1 * coefficient * sin(π/4)
    else # Y goes to Y, X
        new_pauli = 1  # X
        new_pstr = setpauli(pstr, new_pauli, gate.qind)
        new_coefficient = coefficient * sin(π/4)
    end

    add!(output_psum, pstr, coefficient * cos(π/4))
    add!(output_psum, new_pstr, new_coefficient)

    return
end

This should resolve the slight type instability. Let's see if it worked and gives the same results.

In [35]:
@time ourT_psum2 = propagate(ourT_circuit, pstr, thetas; min_abs_coeff=min_abs_coeff);

  0.041896 seconds (24.27 k allocations: 3.164 MiB, 84.09% compilation time: 100% of which was recompilation)


In [36]:
overlapwithzero(ourT_psum2)

0.3326289935884039

In [37]:
ourT_psum == ourT_psum2

true

And check the performance.

In [38]:
@btime propagate($ourT_circuit, $pstr, $thetas; min_abs_coeff=$min_abs_coeff);

  6.475 ms (1154 allocations: 1.64 MiB)


In [39]:
@btime propagate($frozenZ_circuit, $pstr, $thetas; min_abs_coeff=$min_abs_coeff);

  6.083 ms (1158 allocations: 1.79 MiB)


This is already much better and quite fast. But we still see that it is a bit slower than our inbuilt `TGate`. How so? The answer lies in the fact that we move Pauli strings more than necessary. Because the runtime of the T-gate simulation is dominated by commutation (because I is very comon for local observables), we could leave those commuting Pauli strings where they are -> in their original Pauli sum. For this, we can overload the function `applytoall!()`, which differs in that one performs the loop over the Pauli strings in the Pauli sum here, and one can thus use the old Pauli sum more flexibly. Our convention is that anything left in `psum` or `aux_psum` is later merged back into `psum`. Thus, we can simply skip the commuting Pauli strings, and edit the coefficient of Pauli strings in-place. See this version of the function:

In [40]:
function PauliPropagation.applytoall!(gate::CustomTGate, theta, psum, aux_psum; kwargs...)
    
    for (pstr, coefficient) in psum 
    
        pauli = getpauli(pstr, gate.qind)

        if pauli == 0 || pauli == 3  # I or Z commute
            # do nothing
            continue
        end

        if pauli == 1 # X goes to X, -Y
            new_pauli = 2  # Y
            new_pstr = setpauli(pstr, new_pauli, gate.qind)
            new_coefficient = -1 * coefficient * sin(π/4)
        else # Y goes to Y, X
            new_pauli = 1  # X
            new_pstr = setpauli(pstr, new_pauli, gate.qind)
            new_coefficient = coefficient * sin(π/4)
        end

        set!(psum, pstr, coefficient * cos(π/4))
        set!(aux_psum, new_pstr, new_coefficient)
    end
    return
end

In [41]:
@time ourT_psum2 = propagate(ourT_circuit, pstr, thetas; min_abs_coeff=min_abs_coeff);

  0.063213 seconds (23.54 k allocations: 3.222 MiB, 90.11% compilation time: 100% of which was recompilation)


In [42]:
overlapwithzero(ourT_psum2)

0.3326289935884039

In [43]:
ourT_psum == ourT_psum2

true

And check the performance.

In [44]:
@btime propagate($ourT_circuit, $pstr, $thetas; min_abs_coeff=$min_abs_coeff);

  6.059 ms (1158 allocations: 1.79 MiB)


In [45]:
@btime propagate($frozenZ_circuit, $pstr, $thetas; min_abs_coeff=$min_abs_coeff);

  6.057 ms (1158 allocations: 1.79 MiB)


Enjoy defining custom and high-performance gates! 